In [1]:
from coptpy import *
import pandas as pd
import numpy as np
import itertools

env = Envr()
model=env.createModel('trans')
#导入基础数据
buildlimit=8
citylimit=30
ncity=30
cities=["city"+str(i) for i in range(ncity)]
#将数据转化为字典
supply_list=pd.read_csv("data\supply.txt")['supply'].tolist()
demand_list=pd.read_csv("data\demand.txt")['demand'].tolist()
shipcost_list=np.array(pd.read_csv(r"data\trans_dt.txt" , index_col=0 )  ).reshape(-1)

supply=dict(zip(cities,supply_list))
demand=dict(zip (cities,demand_list ) )
shipcost=dict(zip(itertools.product(cities,cities) ,shipcost_list ) )

#导入约束、变量等
vbuild=model.addVars(cities,vtype=COPT.BINARY)
vship=model.addVars(cities,cities,vtype=COPT.INTEGER)

model.addConstr(quicksum(vbuild[city] for city in cities) <=8)
print(model.getAttr(attrname='rows'))
model.addConstrs(vship.sum(i,'*')  <= supply[i]*vbuild[i] for i in cities )
print(model.getAttr(attrname='rows'))
model.addConstrs((vship.sum('*', j) >= demand[j] for j in cities), nameprefix='demand')

model.setObjective(quicksum(shipcost[i]*vship[i] for i in itertools.product(cities,cities) ) ,sense=COPT.MINIMIZE)

model.solve()

#对结果进行分析
# map: num --> model status 
status_map = {
    0:"COPT.UNSTARTED",
    1:"COPT.OPTIMAL",
    2:"COPT.INFEASIBLE",
    3:"COPT.UNBOUNDED",
    4:"COPT.INF_OR_UNB",
    5:"COPT.NUMERICAL",
    6:"COPT.NODELIMIT",
    7:"COPT.TIMEOUT",
    8:"COPT.UNFINISHED",
    9:"COPT.INTERRUPTED"
}
if model.status == COPT.OPTIMAL or \
    model.status in [COPT.TIMEOUT, COPT.NODELIMIT, COPT.INTERRUPTED] and \
    model.hasmipsol:
    print('模型求解状态：{0}'.format(status_map[model.status]))
    print('')
    print('最小总运输成本: {0:.6f}'.format(model.objval))
    print('')
    print("---"*7,"最优仓库选址方案：","---"*7)
    for key in vbuild:
        if vbuild[key].x > 1e-6:
            print('选择{0:6s}建造仓库'.format(key))
    print('')
    print("---"*7,"最优商品运输方案：","---"*7)
    for key in vship:
        if vship[key].x > 1e-6:
            print("{0} --> {1}: {2:6f} 件".format(key[0], key[1], vship[key].x))
else:
  print('Unexpected solution status')

Cardinal Optimizer v7.2.9. Build date Jun 20 2025
Copyright Cardinal Operations 2025. All Rights Reserved

1
31
Model fingerprint: df5a9429

Using Cardinal Optimizer v7.2.9 on Windows
Hardware has 6 cores and 12 threads. Using instruction set X86_NATIVE (1)
Minimizing a MIP problem

The original problem has:
    61 rows, 930 columns and 1860 non-zero elements
    30 binaries and 900 integers

Starting the MIP solver with 12 threads and 32 tasks

Presolving the problem

The presolved problem has:
    61 rows, 930 columns and 1860 non-zero elements
    120 binaries and 810 integers


     Nodes    Active  LPit/n  IntInf     BestBound  BestSolution     Gap   Time
         0         1      --       0  0.000000e+00            --     Inf  0.02s
H        0         1      --       0  0.000000e+00  2.649000e+03 100.00%  0.03s
         0         1      --      32  1.721870e+00  2.649000e+03  99.93%  0.04s
H        0         1      --      32  1.721870e+00  2.597000e+03  99.93%  0.05s
         0 